## 포스트 코로나 시대의 경제 상황을 국민연금 가입자 오픈데이터를 통해 알아보자?!!

- 데이터셋: 공공 데이터 포털
- 형태: 파일데이터 (csv)
- 다운로드: https://www.data.go.kr/data/3046071/fileData.do

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.2f' % x)

%matplotlib inline

plt.rcParams['axes.unicode_minus'] = False 
#plt.rc('font', family='NanumGothic')
plt.rc('font', family='Malgun Gothic')

## 데이터 (국민연금 데이터)

- 2020년 9월 데이터

In [2]:
df = pd.read_csv('data/national_pension.csv')

## 핵심 데이터 column 추출

```
['사업장명', '가입자수', '신규', '상실', '고지금액']
만 추출하여 df_main의 변수에 담아 주세요
```

In [4]:
df_main = df[['사업장명', '가입자수', '신규', '상실', '고지금액']]

In [5]:
df_main.head()

,사업장명,가입자수,신규,상실,고지금액
0,삼성전자,102981,380,423,45766021520
1,현대자동차,68645,936,740,30189521740
2,엘지전자,40385,565,404,17482513180
3,에스케이하이닉스,28314,626,342,12453248240
4,이마트,27413,390,649,6770816160


## 데이터 통계

```
신규, 상실, 가입자수, 고지금액의 평균을 구해 주세요
```

In [7]:
# 신규
df_main['신규'].mean()

1.7862934732931341

In [8]:
# 상실
df_main['상실'].mean()

1.8762255082634272

In [9]:
# 가입자수
df_main['가입자수'].mean()

56.30243701071813

In [10]:
# 고지금액
df_main['고지금액'].mean()

17037918.953102656

## 월급, 연봉 추정

**국민연금 정보로 어떻게 연봉정보를 계산하나요?**


국민연금 보험률은 9%입니다. 쉽게 이야기 하면 급여(신고소득월액)의 9%를 국민연금으로 내는 것입니다. 

하지만 이를 절반으로 나누어 **4.5%는 회사가, 나머지 절반은 개인이 부담하는 구조**입니다. 회사는 급여 외에 추가로 금액을 부담합니다.

국민연금 보험료는 **소득 상한선과 하한선이 설정되어 있어 소득 전체가 아닌 일부 소득에만 부과**됩니다.

이를 역산하면 신고소득월액의 계산이 가능합니다. 하지만 **상한선과 하한선이 설정되어 있어 실제보다 과소계산**될 수 있습니다

[수식]

- 임직원 평균 월급 = 인당고지금액 / 9% * 100%
- 임직원 평균 연봉 = 임직원 평균 월급 * 12개월

[참고 사이트]
- [크레딧잡](https://kreditjob.com/)

In [11]:
(df['고지금액'] / df['가입자수']).head()

0   444412.28
1   439792.00
2   432896.20
3   439826.53
4   246992.89
dtype: float64

```
고지금액 / 가입자수 계산된 값을 인당고지금액 컬럼을 만들어 넣어 주세요
```

In [9]:
df['인당고지금액'] = df['고지금액'] / df['가입자수']

In [13]:
df['인당고지금액'].head()

0   444412.28
1   439792.00
2   432896.20
3   439826.53
4   246992.89
Name: 인당고지금액, dtype: float64

```
- 평균월급 컬럼을 만들어 주세요
- 평균월급 = 인당고지금액 / 9 * 100
```

In [10]:
df['평균월급'] = df['인당고지금액'] / 9 * 100

```
평균 월급의 평균을 구해주세요
평균 월급이 0원은 제외합니다.
```

In [20]:
df.loc[df['평균월급']!=0, '평균월급'].mean()

2874794.438830212

```
- 평균연봉 컬럼을 만들어 주세요
- 평균연봉 = 평균월급 * 12
```

In [12]:
df['평균연봉'] = df['평균월급'] * 12

```
평균 연봉의 평균을 구해주세요
평균 연봉이 0원은 제외합니다.
```

In [21]:
df.loc[df['평균연봉']!=0, '평균연봉'].mean()

34497533.26596099

## 정렬(Order)

### 연봉

- 상위 10개, 하위 10개 기업을 출력하세요

In [18]:
df.sort_values(by='평균연봉', ascending=False).head(10)

,사업장명,가입자수,신규,상실,고지금액,업종코드,업종코드명,광역시코드,시군구코드,읍면동코드,도로명주소,인당고지금액,평균월급,평균연봉
5242,국회사무처,179,1,3,81033300,701201,비주거용 건물 임대업(점포 자기땅),11,560,110,서울특별시 영등포구 의사당대로,452700.00,5030000.00,60360000.00
72074,에스케이씨,19,0,7,8601300,241106,석탄화학계 화합물 및 기타 기초 유기화학 물질제조업,31,140,112,울산광역시 남구 용잠로,452700.00,5030000.00,60360000.00
105164,비티글로벌서비스코리아,13,0,0,5885100,642002,통신 재판매업,11,560,110,서울특별시 영등포구 여의대로,452700.00,5030000.00,60360000.00
121732,송로텍,12,1,0,5432400,749609,그 외 기타 분류 안된 사업 지원 서비스업,11,560,116,서울특별시 영등포구 양평로,452700.00,5030000.00,60360000.00
131959,젠스엠,11,0,0,4979700,321000,기타 반도체 소자 제조업,41,610,250,경기도 광주시 오포읍 오포로,452700.00,5030000.00,60360000.00
46801,쌍용자동차,26,0,0,11770200,999999,BIZ_NO미존재사업장,11,680,101,서울특별시 강남구 테헤란로,452700.00,5030000.00,60360000.00
54110,에이알엠코리아,24,2,0,10864800,722000,응용 소프트웨어 개발 및 공급업,41,135,102,경기도 성남시 분당구 수내로46번길,452700.00,5030000.00,60360000.00
50233,OCI공통,25,0,0,11317500,252901,포장용 플라스틱 성형용기 제조업,11,140,111,서울특별시 중구 소공로,452700.00,5030000.00,60360000.00
75485,파이어아이코리아유한회사,18,1,0,8148600,722000,응용 소프트웨어 개발 및 공급업,11,680,106,서울특별시 강남구 테헤란로,452700.00,5030000.00,60360000.00
107544,라살자산운용,13,0,1,5885100,999999,BIZ_NO미존재사업장,11,560,110,서울특별시 영등포구 국제금융로,452700.00,5030000.00,60360000.00


In [19]:
df.sort_values(by='평균연봉', ascending=True).head(10)

,사업장명,가입자수,신규,상실,고지금액,업종코드,업종코드명,광역시코드,시군구코드,읍면동코드,도로명주소,인당고지금액,평균월급,평균연봉
93146,밀알희망작업장,15,0,0,432000,749604,그 외 기타 분류 안된 사업 지원 서비스업(기타임가공,48,123,125,경상남도 창원시 성산구 용지로,28800.00,320000.00,3840000.00
120729,구립송파구장애인보호작업장,12,0,0,345600,514122,그 외 기타 상품 전문 도매업,11,710,113,서울특별시 송파구 양산로,28800.00,320000.00,3840000.00
114995,상창택시,12,0,1,345600,602201,택시 운송업,47,250,122,경상북도 상주시 영남제일로,28800.00,320000.00,3840000.00
126965,금강운수,11,0,1,316800,602204,시내버스 운송업 / 시외버스 운송업,44,150,116,충청남도 공주시 소학동길,28800.00,320000.00,3840000.00
107932,마포구립장애인직업재활센터,13,0,0,374760,154102,과자류 및 코코아 제품 제조업,11,440,127,서울특별시 마포구 매봉산로,28827.69,320307.69,3843692.31
94041,사회복지법인빅토리아,15,1,0,439560,154101,떡류 제조업,48,121,101,경상남도 창원시 의창구 의안로49번길,29304.00,325600.00,3907200.00
47934,직지드림플러스,26,0,0,768840,242402,치약 비누 및 기타 세제 제조업,43,112,107,충청북도 청주시 서원구 구룡산로372번길,29570.77,328564.10,3942769.23
95299,진양교통,15,0,0,450360,602201,택시 운송업,48,170,440,경상남도 진주시 명석면 남성로,30024.00,333600.00,4003200.00
81840,이리농산물,17,0,0,534460,749609,그 외 기타 분류 안된 사업 지원 서비스업,45,140,117,전라북도 익산시 번영로1길,31438.82,349320.26,4191843.14
50268,삼경택시,25,0,1,791720,602201,택시 운송업,47,250,121,경상북도 상주시 삼백로,31668.80,351875.56,4222506.67


### 신규 채용

- 상위 10개, 하위 10개 기업을 출력하세요

In [20]:
df.sort_values(by='신규', ascending=False).head(10)

,사업장명,가입자수,신규,상실,고지금액,업종코드,업종코드명,광역시코드,시군구코드,읍면동코드,도로명주소,인당고지금액,평균월급,평균연봉
34,쿠팡풀필먼트서비스유한회사,10442,4541,3709,516854740,630903,화물 포장 검수 및 계량 서비스업,11,710,102,서울특별시 송파구 송파대로,49497.68,549974.19,6599690.23
20,쿠팡,14956,2036,893,4176975500,525102,기타 통신 판매업,11,710,102,서울특별시 송파구 송파대로,279284.27,3103158.52,37237902.29
19,쿠팡풀필먼트서비스유한회사,15376,1851,1298,3088280480,630903,화물 포장 검수 및 계량 서비스업,11,710,102,서울특별시 송파구 송파대로,200850.71,2231674.53,26780094.35
53,케이텍맨파워,7437,1677,470,1614012020,749100,고용 알선업,11,650,108,서울특별시 서초구 남부순환로,217024.61,2411384.55,28936614.58
181,고양시청,2896,1556,200,347474120,701201,비주거용 건물 임대업(점포 자기땅),41,281,101,경기도 고양시 덕양구 고양시청로,119984.16,1333157.31,15997887.66
17,국민건강보험공단,16497,1289,181,5998155800,672000,보험 대리 및 중개업,42,130,117,강원도 원주시 삼보로,363590.70,4039896.68,48478760.18
541,진주시청,1115,1115,156,121491000,701201,비주거용 건물 임대업(점포 자기땅),48,170,119,경상남도 진주시 동진로,108960.54,1210672.65,14528071.75
1,현대자동차,68645,936,740,30189521740,341001,화물 자동차 및 특수 목적용 자동차 제조업,11,650,102,서울특별시 서초구 헌릉로,439792.00,4886577.76,58638933.14
40,엘지이노텍,9040,936,209,3487081680,321000,기타 반도체 소자 제조업,11,500,105,서울특별시 강서구 마곡중앙10로,385739.12,4285990.27,51431883.19
36,유베이스,10159,788,1040,1836888320,749939,그 외 기타 분류 안된 전문 과학 및 기술 서비스업,11,170,127,서울특별시 용산구 한강대로38길,180813.89,2009043.23,24108518.82


In [21]:
df.sort_values(by='신규', ascending=True).head(10)

,사업장명,가입자수,신규,상실,고지금액,업종코드,업종코드명,광역시코드,시군구코드,읍면동코드,도로명주소,인당고지금액,평균월급,평균연봉
67828,지로드코리아,20,0,0,5362120,724000,데이터베이스 및 온라인 정보 제공업,11,590,101,서울특별시 동작구 노량진로,268106.00,2978955.56,35747466.67
84251,도원우드,16,0,0,2410660,361002,기타 목재 가구 제조업,30,230,102,대전광역시 대덕구 대화로32번길,150666.25,1674069.44,20088833.33
84250,유타스,16,0,0,4875220,999999,BIZ_NO미존재사업장,27,140,121,대구광역시 동구 율암로,304701.25,3385569.44,40626833.33
84249,우진,16,0,0,2187000,343000,그 외 자동차용 신품 부품 제조업,41,271,106,경기도 안산시 상록구 오리물길,136687.50,1518750.00,18225000.00
84248,동성ENG,16,0,0,3876500,999999,BIZ_NO미존재사업장,31,710,250,울산광역시 울주군 온산읍 우봉강양로,242281.25,2692013.89,32304166.67
84247,루미레즈코리아,16,0,0,6690860,523352,전기용품 및 조명장치 소매업,41,135,109,경기도 성남시 분당구 판교역로,418178.75,4646430.56,55757166.67
84246,석재상사,16,0,0,4766760,512252,주류 도매업,11,320,105,,297922.50,3310250.00,39723000.00
84244,태담,16,0,0,2999520,525200,방문 판매업,41,450,108,경기도 하남시 조정대로,187470.00,2083000.00,24996000.00
84243,모든해상특종손해사정,16,0,0,6686820,749904,손해 사정업,11,140,166,서울특별시 중구 서소문로,417926.25,4643625.00,55723500.00
84242,서울프로아트,16,0,2,5055160,999999,BIZ_NO미존재사업장,11,140,138,서울특별시 중구 필동로,315947.50,3510527.78,42126333.33


### 상실 King!

- 상위 10개, 하위 10개 기업을 출력하세요

In [22]:
df.sort_values(by='상실', ascending=False).head(10)

,사업장명,가입자수,신규,상실,고지금액,업종코드,업종코드명,광역시코드,시군구코드,읍면동코드,도로명주소,인당고지금액,평균월급,평균연봉
34,쿠팡풀필먼트서비스유한회사,10442,4541,3709,516854740,630903,화물 포장 검수 및 계량 서비스업,11,710,102,서울특별시 송파구 송파대로,49497.68,549974.19,6599690.23
244,매그나칩반도체유한회사,2328,8,1454,973043720,321000,기타 반도체 소자 제조업,43,113,137,충청북도 청주시 흥덕구 직지대로307번길,417974.11,4644156.74,55729880.87
19,쿠팡풀필먼트서비스유한회사,15376,1851,1298,3088280480,630903,화물 포장 검수 및 계량 서비스업,11,710,102,서울특별시 송파구 송파대로,200850.71,2231674.53,26780094.35
36,유베이스,10159,788,1040,1836888320,749939,그 외 기타 분류 안된 전문 과학 및 기술 서비스업,11,170,127,서울특별시 용산구 한강대로38길,180813.89,2009043.23,24108518.82
20,쿠팡,14956,2036,893,4176975500,525102,기타 통신 판매업,11,710,102,서울특별시 송파구 송파대로,279284.27,3103158.52,37237902.29
1,현대자동차,68645,936,740,30189521740,341001,화물 자동차 및 특수 목적용 자동차 제조업,11,650,102,서울특별시 서초구 헌릉로,439792.00,4886577.76,58638933.14
377,교원,1540,17,703,471365540,523511,서적 신문 및 잡지류 소매업,11,140,105,서울특별시 중구 을지로,306081.52,3400905.77,40810869.26
4,이마트,27413,390,649,6770816160,521911,기타 대형 종합 소매업,11,200,115,서울특별시 성동구 뚝섬로,246992.89,2744365.47,32932385.66
51,트랜스코스모스코리아,7713,564,648,1401223060,741300,시장 조사 및 여론 조사업,11,170,112,서울특별시 용산구 원효로90길,181670.30,2018558.94,24222707.29
125,비케이알,4026,709,646,399592600,552107,치킨 전문점,11,110,124,서울특별시 종로구 삼봉로,99253.01,1102811.17,13233734.06


In [23]:
df.sort_values(by='상실', ascending=True).head(10)

,사업장명,가입자수,신규,상실,고지금액,업종코드,업종코드명,광역시코드,시군구코드,읍면동코드,도로명주소,인당고지금액,평균월급,평균연봉
67828,지로드코리아,20,0,0,5362120,724000,데이터베이스 및 온라인 정보 제공업,11,590,101,서울특별시 동작구 노량진로,268106.00,2978955.56,35747466.67
83717,바른푸드,16,0,0,2420300,151301,김치류 제조업,26,200,120,부산광역시 영도구 벚꽃길5번길,151268.75,1680763.89,20169166.67
83715,협성부품상사,16,0,0,5366900,503003,자동차 중고 부품 및 내장품 판매업,11,230,105,서울특별시 동대문구 한천로,335431.25,3727013.89,44724166.67
83713,우리기술,16,1,0,4806420,514361,철물 금속 파스너 및 수공구 도매업,41,610,259,경기도 광주시 곤지암읍 경충대로,300401.25,3337791.67,40053500.00
83711,선광모터스,16,0,0,5548540,292400,건설 및 채광용 기계장비 제조업,43,112,320,충청북도 청주시 서원구 현도면 선동2길,346783.75,3853152.78,46237833.33
83710,서룡초등학교,16,0,0,3400240,999999,BIZ_NO미존재사업장,41,461,102,경기도 용인시 처인구 중부대로1281번길,212515.00,2361277.78,28335333.33
83707,지오뷰,16,0,0,4727180,742104,기타 엔지니어링 서비스업,26,380,103,부산광역시 사하구 하신번영로,295448.75,3282763.89,39393166.67
83706,티오로직스,16,0,0,3530860,611002,외항 화물 운송업,11,140,165,서울특별시 중구 청계천로,220678.75,2451986.11,29423833.33
83705,효신로직스,16,1,0,3829120,343000,그 외 자동차용 신품 부품 제조업,47,130,259,경상북도 경주시 외동읍 모화산단길,239320.00,2659111.11,31909333.33
83703,지에이치유통,16,0,0,4093760,521100,슈퍼마켓,11,380,104,서울특별시 은평구 통일로,255860.00,2842888.89,34114666.67


## 순 유입 (신규 - 상실)

- 상위 10개, 하위 10개 기업을 출력하세요

In [24]:
df['순유입'] = df['신규'] - df['상실']

In [25]:
df.sort_values(by='순유입', ascending=False).head(10)

,사업장명,가입자수,신규,상실,고지금액,업종코드,업종코드명,광역시코드,시군구코드,읍면동코드,도로명주소,인당고지금액,평균월급,평균연봉,순유입
181,고양시청,2896,1556,200,347474120,701201,비주거용 건물 임대업(점포 자기땅),41,281,101,경기도 고양시 덕양구 고양시청로,119984.16,1333157.31,15997887.66,1356
53,케이텍맨파워,7437,1677,470,1614012020,749100,고용 알선업,11,650,108,서울특별시 서초구 남부순환로,217024.61,2411384.55,28936614.58,1207
20,쿠팡,14956,2036,893,4176975500,525102,기타 통신 판매업,11,710,102,서울특별시 송파구 송파대로,279284.27,3103158.52,37237902.29,1143
17,국민건강보험공단,16497,1289,181,5998155800,672000,보험 대리 및 중개업,42,130,117,강원도 원주시 삼보로,363590.70,4039896.68,48478760.18,1108
541,진주시청,1115,1115,156,121491000,701201,비주거용 건물 임대업(점포 자기땅),48,170,119,경상남도 진주시 동진로,108960.54,1210672.65,14528071.75,959
34,쿠팡풀필먼트서비스유한회사,10442,4541,3709,516854740,630903,화물 포장 검수 및 계량 서비스업,11,710,102,서울특별시 송파구 송파대로,49497.68,549974.19,6599690.23,832
40,엘지이노텍,9040,936,209,3487081680,321000,기타 반도체 소자 제조업,11,500,105,서울특별시 강서구 마곡중앙10로,385739.12,4285990.27,51431883.19,727
12,한국철도공사,17911,748,120,6423294220,601000,도시철도 운송업,30,110,113,대전광역시 동구 중앙로,358622.87,3984698.55,47816382.63,628
1090,중랑구청일자리창출과,649,649,34,53113320,701201,비주거용 건물 임대업(점포 자기땅),11,260,106,서울특별시 중랑구 봉화산로,81838.71,909318.95,10911827.43,615
636,제니엘시스템,986,633,73,40412880,999999,BIZ_NO미존재사업장,11,170,108,서울특별시 용산구 만리재로,40986.69,455407.71,5464892.49,560


In [26]:
df.sort_values(by='순유입', ascending=True).head(10)

,사업장명,가입자수,신규,상실,고지금액,업종코드,업종코드명,광역시코드,시군구코드,읍면동코드,도로명주소,인당고지금액,평균월급,평균연봉,순유입
244,매그나칩반도체유한회사,2328,8,1454,973043720,321000,기타 반도체 소자 제조업,43,113,137,충청북도 청주시 흥덕구 직지대로307번길,417974.11,4644156.74,55729880.87,-1446
377,교원,1540,17,703,471365540,523511,서적 신문 및 잡지류 소매업,11,140,105,서울특별시 중구 을지로,306081.52,3400905.77,40810869.26,-686
634,연세대학교,987,25,557,143161740,851101,요양병원,11,410,114,서울특별시 서대문구 연세로,145047.36,1611637.28,19339647.42,-532
788,전남대학교,835,12,450,183356360,701201,비주거용 건물 임대업(점포 자기땅),29,170,107,광주광역시 북구 용봉로,219588.46,2439871.72,29278460.68,-438
369,삼성전자서비스씨에스,1568,3,433,395941540,749921,그 외 기타 분류 안된 사업 지원 서비스업,41,117,101,경기도 수원시 영통구 중부대로,252513.74,2805708.19,33668498.30,-430
1482,티맥스에이앤씨,499,6,385,192374440,722000,응용 소프트웨어 개발 및 공급업,41,135,102,경기도 성남시 분당구 황새울로258번길,385519.92,4283554.66,51402655.98,-379
1608,노랑풍선,466,1,379,119269620,630600,여행사업,11,140,161,서울특별시 중구 수표로,255943.39,2843815.45,34125785.41,-378
2154,야마토코리아홀딩스,363,0,361,154730160,271102,제강업,26,440,104,부산광역시 강서구 명지국제2로28번길,426253.88,4736154.27,56833851.24,-361
1957,충남대학교시간강사,394,0,336,58507560,701201,비주거용 건물 임대업(점포 자기땅),30,200,122,대전광역시 유성구 대학로,148496.35,1649959.39,19799512.69,-336
178,에프알엘코리아,2948,90,419,613417940,513121,남녀용 겉옷 및 셔츠 도매업,11,710,102,서울특별시 송파구 올림픽로,208079.36,2311992.84,27743914.07,-329


## 조건 필터링

```
다음의 조건에 맞는 기업 10개를 출력하세요
- 시군구코드 == 140
- 평균월급 300만원 이상
- 가입자수 내림차순 정렬
```

In [27]:
df.loc[(df['시군구코드'] == 140) & (df['평균월급'] > 3000000)].sort_values(by='가입자수', ascending=False).head(10)

,사업장명,가입자수,신규,상실,고지금액,업종코드,업종코드명,광역시코드,시군구코드,읍면동코드,도로명주소,인당고지금액,평균월급,평균연봉,순유입
16,농협은행,16878,121,88,6834230400,659201,국내은행,11,140,170,서울특별시 중구 통일로,404919.45,4499104.95,53989259.39,33
21,우리은행,14156,118,96,5959599100,659201,국내은행,11,140,121,서울특별시 중구 소공로,420994.57,4677717.42,56132609.02,22
22,신한은행,13558,174,48,5690248340,659201,국내은행,11,140,114,서울특별시 중구 세종대로9길,419696.74,4663297.06,55959564.73,126
24,중소기업은행,12774,331,302,5299048540,659201,국내은행,11,140,105,서울특별시 중구 을지로,414830.79,4609231.02,55310772.30,29
29,하나은행,12234,97,151,5299358260,659201,국내은행,11,140,104,서울특별시 중구 을지로,433166.44,4812960.47,57755525.69,-54
49,씨제이제일제당,7792,229,311,2730600020,154200,설탕 제조업,11,140,147,서울특별시 중구 동호로,350436.35,3893737.19,46724846.34,-82
56,기아자동차광주공장,7079,47,15,3189988560,341001,화물 자동차 및 특수 목적용 자동차 제조업,29,140,121,광주광역시 서구 화운로,450627.00,5006966.71,60083600.51,32
59,에스원,6675,18,37,2717108920,319001,운송장비용 조명장치 제조업,11,140,168,,407057.52,4522861.29,54274335.48,-19
82,에스케이텔레콤,5435,25,31,2395914420,642001,무선 및 위성 통신업,11,140,105,서울특별시 중구 을지로,440830.62,4898118.00,58777416.01,-6
84,롯데쇼핑,5419,136,154,1882609780,521910,백화점,11,140,111,서울특별시 중구 남대문로,347409.08,3860100.84,46321210.06,-18


## 중견기업

- **사원수(가입자수) 1000명 이상**인 기업 필터링
- 가입자수 0명 혹은 NaN 제외

In [28]:
people_limit = 1000

In [29]:
big = df.loc[(df['가입자수'].notnull()) & (df['가입자수'] >= people_limit)]

``` 
가입자 순으로 상위 10개 기업을 출력하세요
```

In [30]:
big.sort_values(by='가입자수', ascending=False).head(10)

,사업장명,가입자수,신규,상실,고지금액,업종코드,업종코드명,광역시코드,시군구코드,읍면동코드,도로명주소,인당고지금액,평균월급,평균연봉,순유입
0,삼성전자,102981,380,423,45766021520,323001,비디오 및 기타 영상 기기 제조업,41,117,101,경기도 수원시 영통구 삼성로,444412.28,4937914.28,59254971.33,-43
1,현대자동차,68645,936,740,30189521740,341001,화물 자동차 및 특수 목적용 자동차 제조업,11,650,102,서울특별시 서초구 헌릉로,439792.00,4886577.76,58638933.14,196
2,엘지전자,40385,565,404,17482513180,293001,주방용 전기 기기 제조업,11,560,110,서울특별시 영등포구 여의대로,432896.20,4809957.82,57719493.81,161
3,에스케이하이닉스,28314,626,342,12453248240,300100,컴퓨터 제조업,41,500,253,경기도 이천시 부발읍 경충대로,439826.53,4886961.39,58643536.72,284
4,이마트,27413,390,649,6770816160,521911,기타 대형 종합 소매업,11,200,115,서울특별시 성동구 뚝섬로,246992.89,2744365.47,32932385.66,-259
5,엘지디스플레이,26668,82,141,11137790660,321000,기타 반도체 소자 제조업,11,560,110,서울특별시 영등포구 여의대로,417646.27,4640514.08,55686168.99,-59
6,삼성디스플레이,23143,79,357,10172123300,323001,비디오 및 기타 영상 기기 제조업,41,463,109,경기도 용인시 기흥구 삼성로,439533.48,4883705.32,58604463.84,-278
7,케이티,22495,44,115,9944891340,642001,무선 및 위성 통신업,41,135,103,경기도 성남시 분당구 불정로,442093.41,4912149.04,58945788.49,-71
8,한국전력공사,22426,19,63,9351932300,401000,태양력 발전업,46,170,134,전라남도 나주시 전력로,417012.94,4633477.16,55601725.97,-44
9,홈플러스,21199,98,183,4839358280,522071,기타 음ㆍ식료품 위주 종합 소매업,11,500,102,서울특별시 강서구 화곡로,228282.39,2536470.94,30437651.34,-85


```
- 광역시코드가 11인 기업
- ['사업장명','도로명주소','신규'] 컬럼 필터링
- 신규 인력 순서로 내림차순 정렬
- 10개 기업 출력
```

In [31]:
df.loc[df['광역시코드'] == 11][['사업장명','도로명주소','신규']].sort_values(by='신규', ascending=False).head(10)

,사업장명,도로명주소,신규
34,쿠팡풀필먼트서비스유한회사,서울특별시 송파구 송파대로,4541
20,쿠팡,서울특별시 송파구 송파대로,2036
19,쿠팡풀필먼트서비스유한회사,서울특별시 송파구 송파대로,1851
53,케이텍맨파워,서울특별시 서초구 남부순환로,1677
1,현대자동차,서울특별시 서초구 헌릉로,936
40,엘지이노텍,서울특별시 강서구 마곡중앙10로,936
36,유베이스,서울특별시 용산구 한강대로38길,788
99,프로에스콤,서울특별시 강남구 테헤란로,713
125,비케이알,서울특별시 종로구 삼봉로,709
14,스타벅스커피코리아,서울특별시 중구 소공로,684


## 업종별 신규 유입 현황

```
업종별 신규 유입 인력의 평균을 구한 뒤 
상위 10개 업종을 출력하세요
```

In [32]:
pd.DataFrame(df.groupby('업종코드명')['신규'].mean()).sort_values('신규', ascending=False).head(10)

,신규
업종코드명,
음료품배달원,538.00
자동차용 엔진 제조업,117.00
화물 포장 검수 및 계량 서비스업,89.26
도시철도 운송업,66.50
빵류 제조업,62.25
설탕 제조업,42.29
기타 통신 판매업,42.20
담배제품 제조업,37.00
콜센터 및 텔레마케팅 서비스업,34.00
